In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install SpeechRecognition

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.8/32.8 MB 61.6 MB/s eta 0:00:00


In [ ]:
import torch
from transformers import BertTokenizer, BertModel
import speech_recognition as sr
import numpy as np
import torch.nn as nn
import pandas as pd

In [ ]:
class KoBERTEmotionClassifier(nn.Module):
    def __init__(self, n_classes):
        super(KoBERTEmotionClassifier, self).__init__()
        self.bert = BertModel.from_pretrained("monologg/kobert")
        self.drop = nn.Dropout(p=0.3)
        self.out = nn.Linear(self.bert.config.hidden_size, n_classes)

    def forward(self, input_ids, attention_mask, token_type_ids):
        _, pooled_output = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            return_dict=False
        )
        output = self.drop(pooled_output)
        return self.out(output)

In [ ]:
n_classes = 6  # 감정 클래스 수 (필요에 따라 수정)
model = KoBERTEmotionClassifier(n_classes)
model.load_state_dict(torch.load("/content/drive/MyDrive/saved_kobert_emotion_model.pth"))
model.eval()  # 평가 모드로 설정
tokenizer = BertTokenizer.from_pretrained("/content/drive/MyDrive/saved_tokenizer")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/426 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/369M [00:00<?, ?B/s]

<ipython-input-5-98283734912f>:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("/content/drive/MyDrive/saved_kobert_emotion_model.pth"))


In [ ]:
def transcribe_audio_to_text(audio_file):
    recognizer = sr.Recognizer()
    with sr.AudioFile(audio_file) as source:
        audio_data = recognizer.record(source)
    try:
        text = recognizer.recognize_google(audio_data, language="ko-KR")
        return text
    except sr.UnknownValueError:
        return "STT 실패: 음성을 인식할 수 없습니다."
    except sr.RequestError as e:
        return f"STT 실패: {e}"

In [ ]:
def analyze_sentiment_kobert(text):
    # 텍스트를 KoBERT 모델에 맞게 토큰화 및 텐서 변환
    inputs = tokenizer.encode_plus(
        text,
        None,
        add_special_tokens=True,
        max_length=64,
        padding='max_length',
        return_token_type_ids=True,
        truncation=True,
        return_tensors='pt'
    )

    input_ids = inputs['input_ids']
    attention_mask = inputs['attention_mask']
    token_type_ids = inputs['token_type_ids']

    # 감정 예측
    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        logits = outputs
        predicted_class = torch.argmax(logits, dim=1).item()

    # 감정 레이블 매핑 (사용자 정의 레이블로 변경 가능)
    label_map = {0: "행복", 1: "슬픔", 2: "분노", 3: "중립", 4: "두려움", 5: "혐오"}
    return label_map.get(predicted_class, "알 수 없는 감정")

In [ ]:
def stt_and_sentiment_analysis(audio_file):
    # STT 수행
    text = transcribe_audio_to_text(audio_file)

    # STT 실패 시 메시지 출력
    if "STT 실패" in text:
        return text

    # 감정 분석 수행
    sentiment = analyze_sentiment_kobert(text)

    # 결과 출력
    print(f"STT 결과 텍스트: {text}")
    print(f"감정 분석 결과: {sentiment}")

In [ ]:
df = pd.read_csv('aihub.csv')
df_train = pd.read_csv('data_emotion.csv')
print(df.shape, df_train.shape)

(37196, 2) (14382, 3)


In [ ]:
df_not_in_train = df[~df['Path'].isin(df_train['Path'])]
print(df_not_in_train.shape)

(22814, 2)


In [ ]:
sampled_paths = df_not_in_train['Path'].sample(5).to_list()

for path in sampled_paths:
    print(path)

/content/drive/MyDrive/project/data_aihub/4/5e43f32b2b6c8802f2d85ffd.wav
/content/drive/MyDrive/project/data_aihub/4/5e42c2bb189842034d9f7640.wav
/content/drive/MyDrive/project/data_aihub/4/5e37e400c8c25f16cd145832.wav
/content/drive/MyDrive/project/data_aihub/5/5ef8a16b497740547350bef6.wav
/content/drive/MyDrive/project/data_aihub/4/5e49bc5a7bef803b4851dd3f.wav


In [ ]:
audio_file_path = "/content/drive/MyDrive/project/data_aihub/5_2/5f707eb99e04b149046cbec9.wav"  # 음성 파일 경로
stt_and_sentiment_analysis(audio_file_path)

STT 결과 텍스트: 한번 그렇게 해 볼게 고마워
감정 분석 결과: 행복


In [ ]:
audio_file_path = "/content/drive/MyDrive/project/data_aihub/5/5f07d84eb140144dfcff0cc5.wav"  # 음성 파일 경로
stt_and_sentiment_analysis(audio_file_path)

STT 결과 텍스트: 해피가 갑자기 우리 곁을 떠나고 나니까 해피의 자리가 너무 크게 느껴져 너무 허전해
감정 분석 결과: 슬픔


In [ ]:
audio_file_path = "/content/drive/MyDrive/project/data_aihub/4/5e45d8772a2d173b73a02a08.wav"  # 음성 파일 경로
stt_and_sentiment_analysis(audio_file_path)

STT 결과 텍스트: 고등학교 때 친구인데 이렇게 방을 더럽게 사용할 줄 몰랐어
감정 분석 결과: 분노


In [ ]:
audio_file_path = "/content/drive/MyDrive/project/data_aihub/4/5e351b635807b852d9e077ad.wav"  # 음성 파일 경로
stt_and_sentiment_analysis(audio_file_path)

STT 결과 텍스트: 아니 대학교 와서 처음 만났지
감정 분석 결과: 중립


In [ ]:
audio_file_path = "/content/drive/MyDrive/project/data_aihub/5_2/5f633b67b878131628349eb5.wav"  # 음성 파일 경로
stt_and_sentiment_analysis(audio_file_path)

STT 결과 텍스트: 간신히 잠이 들어서 자고 있는데 갑자기 천둥소리 때문에 놀라서 깼어
감정 분석 결과: 두려움


In [ ]:
audio_file_path = "/content/drive/MyDrive/project/data_aihub/5_2/5fba3a404c55eb78bd7ce2aa.wav"  # 음성 파일 경로
stt_and_sentiment_analysis(audio_file_path)

STT 결과 텍스트: 아 진짜 존나 예민해 짜증 나
감정 분석 결과: 혐오
